In [1]:
import numpy as np 
import math
import matplotlib.pyplot as plt 
from scipy import optimize
from scipy.optimize import fsolve


from sympy import symbols
from sympy.physics.wigner import wigner_3j
import module1 as m1 # module1 contains the function to convert r_star to r\n",

import cmath
from astropy.io import fits
import I_functions_class as Inp
from imp import reload
reload(Inp)

<module 'I_functions_class' from '/users/PCON0003/koivuemily/PrimordialBlackHole/I_functions_class.py'>

In [2]:
 
M = 1.e21 #also in planck units
T = 1/(8*np.pi*M)
#omegas = np.linspace(.01*T,20*T,2000)
omega=5*T

In [ ]:
#Is needed

#I_minplus_inupin(h,omega+h,omega)
#I_plusplus_upupin(h,omega-h,omega)
#I_minplus_upupin(h,omega+h,omega)

#I_plusplus_ininup(h,omega-h,omega)
#I_minplus_inupup(h,omega+h,omega)
#I_plusplus_upinup(h,omega-h,omega)
#I_minplus_upinup(h,omega+h,omega)

#I_minplus_inupup(h,omega+h,omega)  -> repeated
#I_minplus_inupin(h,omega+h,omega)  -> repeated
#I_plusplus_upupup(h,omega-h,omega)
#I_plusplus_upupin(h,omega-h,omega) -> repeated
#I_minplus_upupup(h,omega+h,omega)
#I_minplus_upupin(h,omega+h,omega)  -> repeated

#I_plusplus_ininup(h,omega-h,omega) -> repeated
#I_plusplus_ininin(h,omega-h,omega) 
#I_plusplus_upinup(h,omega-h,omega) -> repeated
#I_plusplus_upinin(h,omega-h,omega)
#I_minplus_upinup(h,omega+h,omega)  -> repeated
#I_minplus_upinin(h,omega+h,omega)



In [ ]:
#I_minplus_inupin(h,omega+h,omega)

In [19]:
X = 0
X_prime = 1
X_gamma = 0
l=1
parity = 1
n=1
lim1=0
lim2=1

ks=[-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
k_primes=[-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
omega_index = round(omega*100*(8*np.pi*M) -1)

In [20]:
hduX = fits.Header()
hduX['type']='IFunc'
hduX['parity'] = 'Even'
hduX['Subs'] = 'IN UP IN'
hduX['M']= M
hduX['omega']= omega
hduX['l']= 1
empty_primary = fits.PrimaryHDU(header=hduX)
hdulX = fits.HDUList([empty_primary])

#ok so this is for a specific omega 
# need to span hs, and need to have different k,k',l


#only 5 l, so can be diff file for each l
#so each table is an h, and the column is k and row is k' 



In [5]:
#Emax = 20T

h_s = np.linspace(.01*T,20*T,2000)

In [6]:
h_s[500-4]/T

4.97

In [7]:
h_prime_allowed = h_s[501:]
h = np.linspace(.02*T,20*T,1999)
print(h_prime_allowed[0]/T)
vals = np.zeros((len(ks),len(k_primes)),dtype=complex)


h_in = round(h[0]*100*(8*np.pi*M) -1)
print(h_in)

5.02
1


In [21]:
direcPhoton = '/fs/scratch/PCON0003/emily/PhotonWaveFunctionFits/'
hduP = fits.open(direcPhoton+str(l)+'.fits')
        
r_points_gamma = hduP[omega_index].data.field('rpoints_up')
 
psi_gammalomega = hduP[omega_index].data.field('F_points_in')
psi_gammalomega_prime = hduP[omega_index].data.field('z_points_in')
        

In [22]:
for x in range(len(h_prime_allowed)):
    print(h[x]/T)
    for k in range(len(ks)):
        print(k)
        j=(np.abs(ks[k]) - 1/2)
   
        for k_prime in range(len(k_primes)):
            j_prime = (np.abs(k_primes[k_prime]) - 1/2)
            tryA2=Inp.IfunctionsNoM(X,ks[k],X_prime,k_primes[k_prime],X_gamma,l,parity,h[x],h_prime_allowed[x],omega,M,n)
                    
            if np.abs(tryA2.coeff_no_m_even)>1.e-10:
                if (-1)**(ks[k]+k_primes[k_prime]+l)>0 and ks[k]*k_primes[k_prime]>0:
                    print('first')
                    #tryA2=Inp.IfunctionsNoM(X,ks[k],X_prime,k_primes[k_prime],X_gamma,l,parity,h[x],h_prime_allowed[x],omega,M,n)
                    vals[k][k_prime] = tryA2.IBarminusplusevenfunc(X,ks[k],X_prime,k_primes[k_prime],psi_gammalomega,psi_gammalomega_prime,h[x],l,h_prime_allowed[x],omega,M,lim1,lim2)
                if (-1)**(ks[k]+k_primes[k_prime]+l)<0 and ks[k]*k_primes[k_prime]<0:
                    print('second')
                    #tryA2=Inp.IfunctionsNoM(X,ks[k],X_prime,k_primes[k_prime],X_gamma,l,parity,h[x],h_prime_allowed[x],omega,M,n)
                    vals[k][k_prime] = tryA2.IBarminusplusevenfunc(X,ks[k],X_prime,k_primes[k_prime],psi_gammalomega,psi_gammalomega_prime,h[x],l,h_prime_allowed[x],omega,M,lim1,lim2)

    col0 = fits.Column(name='k val',format='D',array=ks)
    col1 = fits.Column(name='k_prime -10',format='M',array=vals[:,0])
    col2 = fits.Column(name='k_prime -9',format='M',array=vals[:,1])
    col3 = fits.Column(name='k_prime -8',format='M',array=vals[:,2])
    col4 = fits.Column(name='k_prime -7',format='M',array=vals[:,3])
    col5 = fits.Column(name='k_prime -6',format='M',array=vals[:,4])
    col6 = fits.Column(name='k_prime -5',format='M',array=vals[:,5])
    col7 = fits.Column(name='k_prime -4',format='M',array=vals[:,6])
    col8 = fits.Column(name='k_prime -3',format='M',array=vals[:,7])
    col9 = fits.Column(name='k_prime -2',format='M',array=vals[:,8])
    col10 = fits.Column(name='k_prime -1',format='M',array=vals[:,9])
    col11 = fits.Column(name='k_prime 1',format='M',array=vals[:,10])
    col12 = fits.Column(name='k_prime 2',format='M',array=vals[:,11])
    col13 = fits.Column(name='k_prime 3',format='M',array=vals[:,12])
    col14 = fits.Column(name='k_prime 4',format='M',array=vals[:,13])
    col15 = fits.Column(name='k_prime 5',format='M',array=vals[:,14])
    col16 = fits.Column(name='k_prime 6',format='M',array=vals[:,15])
    col17 = fits.Column(name='k_prime 7',format='M',array=vals[:,16])
    col18 = fits.Column(name='k_prime 8',format='M',array=vals[:,17])
    col19 = fits.Column(name='k_prime 9',format='M',array=vals[:,18])
    col20 = fits.Column(name='k_prime 10',format='M',array=vals[:,19])
    
    
    
    cols = fits.ColDefs([col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20])
    table_hdu = fits.BinTableHDU.from_columns(cols)
    table_hdu.header['h']=h[x]
    
    hdulX.append(table_hdu)


0.02
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.03
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.04
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.06
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.06999999999999999
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.08
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.09000000000000001
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.11
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.12
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.13999999999999999
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.18000000000000002
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.19
0

In [24]:
hdulX.writeto('IFuncFits/minusplus_inupin_1.fits')

In [ ]:
#i think i need to impliment the delta selection rules at this step 
#ok cut down to each file needing 916 hours which seems still bad 
#this is gonna take wayyy too long


#can try to also use the triangle inequality step 